In [ ]:
import pandas as pd

import tensorflow as tf
import tensorflow_hub as tf_hub
import wandb

import tensorflow as tf
import tensorflow_text
import nltk
from lime.lime_text import LimeTextExplainer
import numpy as np
from train_wandb import Trainer

In [ ]:
labeled_df = pd.read_excel("../../data/labeled_dataset.xlsx").rename(columns={"Unnamed: 0": "id"})
minified_df = labeled_df[["sentence", "Label_opinion"]]
minified_df["target"] = (minified_df["Label_opinion"] == "Expresses writer’s opinion").astype(int)
minified_df = minified_df.drop("Label_opinion", axis=1)
minified_df.head()

In [ ]:
mpqa_df = pd.read_csv("../../data/mpqa_filtered.csv", sep = ";").rename(columns={"sent": "sentence"})
mpqa_df["target"] = np.where(mpqa_df['score'] >= 2, 1, 0)
mpqa_df.drop(columns=["score"], inplace = True)
mpqa_df.head()

In [ ]:
full_dataset_df = pd.concat([minified_df, mpqa_df])
dataset = (
    tf.data.Dataset.from_tensor_slices(
        (full_dataset_df['sentence'].values, full_dataset_df['target'].values)
    )
    .shuffle(full_dataset_df.shape[0])
)

In [ ]:
no_opinion_text = """
Natural language processing (NLP) is an interdisciplinary subfield of linguistics, computer science, 
and artificial intelligence concerned with the interactions between computers and human language, 
in particular how to program computers to process and analyze large amounts of natural language data. 
The goal is a computer capable of understanding the contents of documents, 
including the contextual nuances of the language within them. The technology can then accurately extract 
information and insights contained in the documents as well as categorize and organize the documents themselves.
"""

In [ ]:
opinion_text = """
One of my favourite films is Titanic. I`m a great fan of romantic movies and I`m very keen on the history of the tragic Titanic. 
The movie tells the dramatic story of the Titanic with Leonardo Di Caprio and Kate Winslet as the main actors.
In the year 1912 a young poor guy, Leonardo, travels by the gorgeous ship Titanic from London to New York with a big dream. 
On board he meets a fabulous, wealthy girl, Kate Winslet. He fells in love with her as soon as he sees this gorgeous girl. 
However, after a few days the ship hits an enormous iceberg and the tragedy begins to unfold. The Titanic starts sinking…
The movie is very close to the real tragedy. It shows a beautiful love story with an extremely sad ending. 
The acting is first-rate. Leonardo Di Caprio`s and Kate Winslet`s performance is so brilliant that I cried during the whole movie.
This movie brings a tear to your eyes. If you want to cry and melt down to a glamurous love story, 
I strongly recommend that you watch this movie. Titanic is well worth seeing.
"""

In [ ]:
def build_model(preprocess_layer, encoder_layer):
    input_layer = tf.keras.layers.Input(shape=(), dtype=tf.string, name="input")
    preprocessing_layer = preprocess_layer(input_layer)
    encoder_outputs = encoder_layer(preprocessing_layer)
    pooled_output = encoder_outputs["pooled_output"]
    output_layer = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(pooled_output)
    model = tf.keras.Model(input_layer, output_layer)
    return model

In [ ]:
def make_experiment(config, name):
    
    preprocess_layer = tf_hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
        name="preprocessing"
    )

    encoder_layer = tf_hub.KerasLayer(
        "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/2",
        name="BERT_encoder",
        trainable=True
    )

    model = build_model(preprocess_layer, encoder_layer)
    model.summary()

    trainer = Trainer(config,model,dataset, name)
    trainer.compile_model()
    trainer.train()
    trainer.save_model()
    trainer.test_model()
    trainer.predict_model(no_opinion_text, "no opinion text")
    trainer.predict_model(opinion_text, "opinion text")

    def get_probabilities(text):
        prob = model.predict([text])
        return np.hstack([1 - prob, prob])

    explainer = LimeTextExplainer(class_names=["Non-biased", "Biased"])
    explanation_2 = trainer.explain_prediction(no_opinion_text, "No opinion text explanation")
    explanation_1 = trainer.explain_prediction(opinion_text,  "Opinion text explanation")
    with tf.device("/cpu:0"):
        explanation_1.show_in_notebook(text=True)
        explanation_2.show_in_notebook(text=True)
    wandb.finish()


In [ ]:
config = {
    "architecture": "bigger-BERT-CNN",
    "dataset": "full_dataset",
    "datasize": minified_df.shape[0],
    "epochs": 50,
    "steps_per_epoch": tf.data.experimental.cardinality(dataset).numpy(),
    "num_train_steps": tf.data.experimental.cardinality(dataset).numpy() * 20,
    "num_warmup_steps": int(0.1 * tf.data.experimental.cardinality(dataset).numpy() * 20),
    "init_lr": 3e-5,
    "batch_size" : 1
}

In [ ]:
batch_table = [1,2,4,8,16,32,64]

In [ ]:
lr_table = [3e-6, 1e-5, 1e-4]

In [ ]:
make_experiment(config, f"Experiment-bigger-BERT")